# XMHW tests on the OFAM3 dataset

Purpose
-------
    The following will investigate the capability of xmhw to parallelise the MHW analysis on a subset of temperature data from the OFAM3 - 10th degree resolution global simulation from 1980-2100. The simulation runs from 1980 to 2006 under JRA55 atmospheric forcing, and thereafter the reanalysis is repeated but with the addition of the RCP8.5 climate trend.

    Contents:
        1. Load in Temperature Data and visualise (2D in space, 1D in time)
        2. Select the region around Australia to perform the heatwave analysis and throw rest away
        3. Calculate the climatology required for the heatwave analysis and save as a new netcdf file
            [ this will be read in later and in a new session for performing the heatwave analysis ]
        4. Perform heatwave analysis using xmhw by iterating around the subsetted grid

Thanks to John Reilly for sharing his [code](https://github.com/Thomas-Moore-Creative/shared_sandbox/blob/main/mhw-3d-scalingTests-gadiJup.ipynb)
    


 some sandbox edits here from Thomas Moore - 27 April 2024

### imports

In [1]:
import sys
import os

### data handling
import numpy as np
import pandas as pd
import xarray as xr
import scipy as sci

### plotting
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.gridspec import GridSpec
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean.cm as cmo
from cmocean.tools import lighten

### marine heatwaves python package
from xmhw.xmhw import threshold, detect

# print versions of packages
print("python version =",sys.version[:5])
print("numpy version =", np.__version__)
print("pandas version =", pd.__version__)
print("xarray version =", xr.__version__)
print("scipy version =", sci.__version__)
print("matplotlib version =", sys.modules[plt.__package__].__version__)
print("cmocean version =", sys.modules[cmo.__package__].__version__)
print("cartopy version =", sys.modules[ccrs.__package__].__version__)

python version = 3.10.
numpy version = 1.26.4
pandas version = 2.2.1
xarray version = 2024.3.0
scipy version = 1.12.0
matplotlib version = 3.8.3
cmocean version = v3.0.3
cartopy version = 0.22.0


### import the dask client for assessing performance

In [2]:
from dask.distributed import Client
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 251.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39755,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.18 GiB
Comm: tcp://127.0.0.1:36981,Total threads: 1
Dashboard: /proxy/38595/status,Memory: 8.97 GiB
Nanny: tcp://127.0.0.1:36469,


## grab the future temperature data from fp2

In [7]:
wrkdir = "/g/data/fp2/OFAM3" #not using os.chdir() 

### what is native chunking ???
```
/g/data/fp2/OFAM3/jra55_rcp8p5/surface du -hs ocean_temp_sfc_2050_03.nc
320M	ocean_temp_sfc_2050_03.nc

short temp(Time, st_ocean, yt_ocean, xt_ocean) ;
		temp:long_name = "Potential temperature" ;
		temp:units = "degrees C" ;
		temp:valid_range = -32767s, 32767s ;
		temp:missing_value = -32768s ;
		temp:_FillValue = -32768s ;
		temp:packing = 4 ;
		temp:scale_factor = 0.001678518f ;
		temp:add_offset = 45.f ;
		temp:cell_methods = "time: mean" ;
		temp:time_avg_info = "average_T1,average_T2,average_DT" ;
		temp:coordinates = "geolon_t geolat_t" ;
		temp:standard_name = "sea_water_potential_temperature" ;

```
#### `du -hs` reveals no chunking information. Is temp a single chunk? Short variable is 320MB so bigger float value expected once loaded

In [21]:
# preprocesser to drop unwanted variables
def drop_stuff(ds, coords_to_drop,vars_to_drop):
    """
    Preprocessor function to drop specified coordinates and variables from a dataset loaded via xr.open_mfdataset

    Parameters:
        ds (xarray.Dataset): The dataset from which coordinates & variables are to be dropped.
        coords_to_drop (list of str): List of coordinate names to drop.
        vars_to_drop(list of str): List of variable names to drop

    Returns:
        xarray.Dataset: Dataset with specified coordinates and variables dropped.
    """
    # Drop coordinates if they are in the dataset
    ds = ds.drop_vars(coords_to_drop, errors='ignore')
    ds = ds.drop_vars(vars_to_drop, errors='ignore')
    return ds

In [22]:
%%time
coords_to_drop =['st_edges_ocean','nv']
vars_to_drop =['Time_bounds','average_DT','average_T1','average_T2']
sst = xr.open_mfdataset(wrkdir + "/jra55_rcp8p5/surface/ocean_temp_sfc_205*.nc", parallel=True,preprocess=lambda x: drop_stuff(x, coords_to_drop,vars_to_drop)).squeeze() #combine='by_coords' is default
sst

CPU times: user 1.04 s, sys: 103 ms, total: 1.14 s
Wall time: 1.16 s


<xarray.Dataset> Size: 79GB
Dimensions:   (Time: 3652, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * Time      (Time) datetime64[ns] 29kB 2050-01-01T12:00:00 ... 2059-12-31T1...
    st_ocean  float64 8B 2.5
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
Data variables:
    temp      (Time, yt_ocean, xt_ocean) float32 79GB dask.array<chunksize=(31, 1500, 3600), meta=np.ndarray>
Attributes:
    filename:       TMP/ocean_ofam_2050_01.nc.0000
    NumFilesInSet:  1350
    title:          jra_55_1979
    grid_type:      regular
    history:        Wed Jun 17 01:58:51 2015: ncks -d st_ocean,0 /short/fp2/m...
    NCO:            4.3.8

In [ ]:
print("Future SST dataset = %i Gb"%(sst.nbytes/1e9))

## fetch the climatology and threshold that were calculated in the other notebook

In [ ]:
os.chdir("/g/data/es60/pjb581/heatwaves")
climatology = xr.open_dataset('Australian_SST_daily_climatology.nc')['temp']
threshold90 = xr.open_dataset('Australian_SST_daily_MHWthreshold.nc')['temp']

threshold90


## detect MHWs

In [ ]:
sst1 = sst.copy(deep=True).chunk({'time':-1, 'yt_ocean':50, 'xt_ocean':50})

print("Adding 'year' as a new coordinate") 
sst1['year'] = sst1['time'].dt.year

print("Adjust for proper day indexing in leap years")
def adjust_dayofyear(times):
    return np.array([pd.Timestamp(t).dayofyear for t in times])

print("Adding 'doy' as a new coordinate")
print(" This is essential because we need to be able to compare the threshold with dimensions ('doy', 'lat', 'lon') to the new SST dataset")
print(" i.e., at each day of the year, we see if the SST is greater than the threshold")
sst1['doy'] = (('time',), adjust_dayofyear(sst1['time'].values))

sst1 = sst1.set_index(time=['year', 'doy'])
sst1


In [ ]:
print("Detect MHWs")

mhws = {}
diff = {}

for yr in np.unique(sst1['year'].values):
    print(yr)
    tmp = sst1.sel(year=yr)
    if (yr == 2052) or (yr == 2056):
        print("leap")
        mhws['%i'%(yr)] = tmp > threshold90
        diff['%i'%(yr)] = tmp - threshold90
    else:
        mhws['%i'%(yr)] = tmp > threshold90
        diff['%i'%(yr)] = tmp - threshold90
    mhws['%i'%(yr)] = mhws['%i'%(yr)].rename({"doy":"time"}).assign_coords(time = pd.date_range(start=f"{yr}-01-01", end=f"{yr}-12-31", freq='D'))
    diff['%i'%(yr)] = diff['%i'%(yr)].rename({"doy":"time"}).assign_coords(time = pd.date_range(start=f"{yr}-01-01", end=f"{yr}-12-31", freq='D'))

print("Concatenate the years")
tmp = list(mhws.values())
mhws = xr.concat(tmp, dim='time')

tmp = list(diff.values())
diff = xr.concat(tmp, dim='time')

del tmp

In [ ]:
plt.figure()
plt.pcolormesh(mhws.mean(dim='time'))
plt.colorbar()

plt.figure()
plt.pcolormesh(diff.isel(time=365), cmap = cmo.balance, vmin=-2, vmax=2)
plt.colorbar()



## Now apply a rolling mean on the data in time to determine where MHWs occur
    If the sum over a window of x days is at least x, it indicates that the SST was above the threshold for x consecutive days.

In [ ]:
%%time

### set minimum duration of a heatwave
min_duration = 10
mhws_occurance = mhws.rolling(time=min_duration, center=False).sum() >= min_duration

plt.figure()
plt.pcolormesh(mhws_occurance.mean(dim='time'))
plt.colorbar()

plt.figure()
plt.plot(mhws.coords['time'], mhws.isel(xt_ocean=100,yt_ocean=100))
plt.plot(mhws_occurance.coords['time'], mhws_occurance.isel(xt_ocean=100,yt_ocean=100))



## create an event label data array
    all events at each point in space are numbered from 1 onwards

In [ ]:
%%time
# Label continuous events in time
def label_events(da):
    shifted = da.shift(time=1, fill_value=False)
    new_event_start = da & ~shifted
    event_labels = new_event_start.cumsum(dim='time')
    return event_labels.where(da)

event_labels = label_events(mhws_occurance).compute()


In [ ]:
plt.figure()
plt.pcolormesh(event_labels.isel(time=365))
plt.colorbar()

plt.figure()
plt.pcolormesh(diff.isel(time=365), cmap = cmo.balance, vmin=-2, vmax=2)
plt.colorbar()


In [ ]:
%%time

# Find maximum number of events in the dataset
max_event_number = int(event_labels.max().values)
print("Max number of events over the grid =", max_event_number)

# Create a new Dataset with the event dimension set to 71
mhws_ds = xr.Dataset({
    'duration': xr.DataArray(
        np.nan, 
        dims=('event', 'yt_ocean', 'xt_ocean'),
        coords={
            'event': np.arange(1, max_event_number+1),  # Using 1-based indexing for events
            'yt_ocean': event_labels.coords['yt_ocean'].values,
            'xt_ocean': event_labels.coords['xt_ocean'].values
        }
    ),
    'intensity_mean': xr.DataArray(
        np.nan, 
        dims=('event', 'yt_ocean', 'xt_ocean'),
        coords={
            'event': np.arange(1, max_event_number+1),  # Using 1-based indexing for events
            'yt_ocean': event_labels.coords['yt_ocean'].values,
            'xt_ocean': event_labels.coords['xt_ocean'].values
        }
    ),
    'intensity_max': xr.DataArray(
        np.nan, 
        dims=('event', 'yt_ocean', 'xt_ocean'),
        coords={
            'event': np.arange(1, max_event_number+1),  # Using 1-based indexing for events
            'yt_ocean': event_labels.coords['yt_ocean'].values,
            'xt_ocean': event_labels.coords['xt_ocean'].values
        }
    )
})
mhws_ds


### calculate the duration of the MHWs

In [ ]:
%%time

for event_id in range(1, max_event_number+1):
    if event_id%5 == 0:
        print(event_id)
    event_mask = event_labels == event_id
    event_duration = event_mask.sum(dim='time')  # Counts the days per event
    
    # Place the calculated duration into the correct position in the new dataset
    mhws_ds['duration'].loc[{'event': event_id}] = event_duration



In [ ]:
diff = diff.compute()


### Calculate the mean intensity of the MHWs

In [ ]:
%%time

for event_id in range(1, max_event_number+1):
    if event_id%5 == 0:
        print(event_id)
    event_meanintensity = diff.where(event_labels == event_id).mean(dim='time')
    
    # Place the calculated mean intensity into the correct position in the new dataset
    mhws_ds['intensity_mean'].loc[{'event': event_id}] = event_meanintensity


### Calculate the maximum intensity of the MHWs

In [ ]:
%%time

for event_id in range(1, max_event_number+1):
    if event_id%5 == 0:
        print(event_id)
    event_maxintensity = diff.where(event_labels == event_id).max(dim='time')
    
    # Place the calculated max intensity into the correct position in the new dataset
    mhws_ds['intensity_max'].loc[{'event': event_id}] = event_maxintensity


In [ ]:
plt.figure()
mhws_ds['duration'].isel(event=0).plot()
plt.figure()
mhws_ds['intensity_mean'].isel(event=0).plot()
plt.figure()
mhws_ds['intensity_max'].isel(event=0).plot()


## save to disk

In [ ]:
%%time
os.chdir("/g/data/es60/pjb581/heatwaves")
os.getcwd()

print("Saving mhws_ds to disk")
mhws_ds.to_netcdf('Australian_MHWs_2050s.nc', mode='w')
